$${\color{yellow}{\text{Applied Linear Algebra: Signal Processing for Machine Learning}}}$$



In [2]:
import pandas as pd
import numpy as np
import numpy.linalg as linalg
import os
import sys
import matplotlib.pyplot as plt
import matplotlib.cm as cm
plt.style.use('dark_background')
%matplotlib inline

In [3]:
## Mount Google drive folder if running in Colab
if('google.colab' in sys.modules):
    from google.colab import drive
    drive.mount('/content/drive', force_remount = True)
    DIR = '/content/drive/My Drive/Colab Notebooks/MAHE/MSIS Coursework/OddSem2025MAHE/Share/Applied Linear Algebra/Notebooks'
    DATA_DIR = DIR + '/Data/'
    os.chdir(DIR)
else:
    DATA_DIR = '../data/'

In [ ]:
## Define and plot a continuos sinusoid along with its sampled discrete version
# Continuous sinusoid properties
f = 5 # cycle frequency
omega = 2 * np.pi * f # angular frequency
x = lambda t: np.sin(omega*t)

# Range of time for plotting
t_start = 0 # start time in sec
t_end = 2 # end time in sec

# Sampling information for creating the discrete sinusoid
fs = 3.0*f # sampling frequency
Ts = 1 / fs # sampling period

fig, ax = plt.subplots(1, 1, figsize = (4, 4))
ax.plot(np.arange(t_start, t_end+1e-03, 1e-03), x(np.arange(t_start, t_end+1e-03, 1e-03)), 'w-', label = 'Continuous Sinusoid')
ax.stem(np.arange(t_start, t_end+1e-10, Ts), x(np.arange(t_start, t_end+1e-10, Ts)), 'r-', label = 'Discrete Sinusoid')
ax.set_xlabel('t (seconds)')
ax.set_ylabel('x(t)')
ax.set_xticks(np.arange(t_start, t_end+1e-01, 1e-01))
ax.legend(loc = 'upper right');

In [7]:
def plotveccomp(x, name = ' ', color = 'black', marker = '*', axis = None):
  ax = axis
  component_index = range(0, len(x))
  ax.plot(component_index, x, color = color, marker = marker)
  ax.plot(component_index, [np.mean(x)]*len(x), linewidth = 1, linestyle = 'dashed', color ='blue')
  ax.plot(component_index, [np.mean(x) - np.std(x)]*len(x), linewidth = 1, linestyle = 'dashed', color ='red')
  ax.plot(component_index, [np.mean(x) + np.std(x)]*len(x), linewidth = 1, linestyle = 'dashed', color ='red')
  ax.set_xlabel('t (seconds)')
  ax.set_ylabel('Voltage (millivolts)')
  ax.set_title(name)

In [ ]:
## Create a discrete signal that we will analyze using the fourier transform
# First create a continuous sinusoid which we will sample
f1 = 1 # cycle frequency
omega1 = 2 * np.pi * f1 # angular frequency
f2 = 2 # cycle frequency
omega2 = 2 * np.pi * f2 # angular frequency
f3 = 4 # cycle frequency
omega3 = 2 * np.pi * f3 # angular frequency
# Function definition for the continuous sinusoid
f = lambda t: 4*np.sin(omega1*t) + 10*np.sin(omega2*t) + 6*np.sin(omega3*t)

# Sampling information for creating the discrete sinusoid
fs = 4*f3 # sampling frequency (should be greater than 2 times the highest frequency in the signal)
Ts = 1 / fs # sampling period

# Range of time for creating the discrete sinusoid
t_start = 0.0 # start time in sec
t_end = 1.0 # end time in sec

# Create the discrete sinuosid by sampling the continuous sinusoid
x = f(np.arange(t_start, t_end+Ts, Ts))

# Plot the continuous and discrete sinusoid
fig, ax = plt.subplots(1, 1, figsize = (5, 5))
fig.tight_layout(pad = 4.0)
ax.plot(np.arange(t_start, t_end+1e-03, 1e-03), f(np.arange(t_start, t_end+1e-03, 1e-03)), 'w-', label = 'Continuous Sinusoid')
ax.stem(np.arange(t_start, t_end+1e-10, Ts), x, 'r-', label = 'Sampled Sinusoid')
ax.legend(loc = 'upper right')
ax.set_xlabel('t (seconds)');

In [ ]:
## Note that the sampled discrete sinusoid x can be treated as a vector.

# Compute the discrete Fourier transform of x using the in-built
# numpy function fft()
X = np.fft.fft(x)
print(X)

# Plot the magnitude of the discrete Fourier transform vector
fig, ax = plt.subplots(1, 1, figsize = (4,4))
fig.tight_layout(pad = 4.0)
ax.stem(range(0, len(X)), np.abs(X), 'w-')
ax.set_xlabel('Frequency (Hz)');

In [ ]:
## Compute the discrete Fourier transform of x from scratch
n = len(x)

# Initialize discrete Fourier transform matrix to zeros
F = np.zeros((n, n))

# Fill-in the entries of the F matrix column by column
w = np.exp(-1j * ((2*np.pi)/n))
F = w**(np.arange(n) * np.arange(n)[:, np.newaxis])

# Compute the discrete Fourier transform
X = np.dot(F, x)
print(X)

# Plot the magnitude of the discrete Fourier transform vector
fig, ax = plt.subplots(1, 1, figsize = (6,6))
fig.tight_layout(pad = 4.0)
ax.stem(range(0, len(X)), np.abs(X), 'w-')
ax.set_xlabel('Frequency (Hz)');

In [ ]:
F[:, 1]

In [ ]:
## Plot the discrete sinusoids corresponding to the real part of each column of F

# Note that the discrete sinusoid x is simply a linear combination of the discrete
# sinusoids that form the columns of the matrix F
for j in range(n):
  fig, ax = plt.subplots(1, 1, figsize = (4, 4))
  fig.tight_layout(pad = 4.0)
  ax.stem(range(n), np.real(F[:, j]), 'r-')

In [ ]:
## A full example using synthetic heart rate data
fs = 100  # Sampling frequency (100 samples per second)
t = np.arange(0, 10, 1/fs)  # Time vector for 10 seconds

# Create a signal composed of two frequencies (0.5 Hz and 3 Hz)
heart_rate_signal = 1.5 * np.sin(2 * np.pi * 0.5 * t) + 0.5 * np.sin(2 * np.pi * 3 * t)

# Add some random noise
heart_rate_signal += 0.3 * np.random.randn(len(t))

# Plot the time-domain signal
plt.figure(figsize=(12, 4))
plt.plot(t, heart_rate_signal, color='royalblue')
plt.title('Synthetic Heart Rate Signal (Time Domain)')
plt.xlabel('Time (seconds)')
plt.ylabel('Amplitude')
plt.show()

# Perform FFT
N = len(heart_rate_signal)  # Number of samples
fft_values = np.fft.fft(heart_rate_signal)  # Compute FFT
fft_frequencies = np.fft.fftfreq(N, 1/fs)  # Frequency bins

# Get magnitude and focus on positive frequencies
fft_magnitude = np.abs(fft_values[:N // 2])
fft_frequencies = fft_frequencies[:N // 2]

# Plot the frequency-domain representation
plt.figure(figsize=(4, 4))
plt.plot(fft_frequencies, fft_magnitude, color='darkorange')
plt.title('Frequency Domain (FFT)')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.grid(True)
plt.show()


In [ ]:
## Extract features from the frequency domain

# Find the dominant frequency
dominant_frequency_index = np.argmax(fft_magnitude)
dominant_frequency = fft_frequencies[dominant_frequency_index]
print(f"Dominant Frequency: {dominant_frequency:.2f} Hz")

# Compute power spectral density (sum of squared magnitudes which indicates how
# the signal power is distributed across different frequency components
power_spectral_density = np.sum(fft_magnitude ** 2)
print(f"Power Spectral Density: {power_spectral_density:.2f}")

# Compute power in low and high-frequency bands
lf_band = (fft_frequencies >= 0.04) & (fft_frequencies <= 0.15)
hf_band = (fft_frequencies >= 0.15) & (fft_frequencies <= 0.4)

lf_power = np.sum(fft_magnitude[lf_band] ** 2)
hf_power = np.sum(fft_magnitude[hf_band] ** 2)

# Compute LF/HF ratio
lf_hf_ratio = lf_power / hf_power if hf_power != 0 else np.inf
print(f"LF Power: {lf_power:.2f}, HF Power: {hf_power:.2f}, LF/HF Ratio: {lf_hf_ratio:.2f}")

# Normalize the magnitude to a probability distribution
fft_power_norm = fft_magnitude / np.sum(fft_magnitude)

# Compute spectral entropy (how evenly the power is distributed across
# frequencies)
spectral_entropy = -np.sum(fft_power_norm * np.log2(fft_power_norm + 1e-10))  # Add small value to avoid log(0)
print(f"Spectral Entropy: {spectral_entropy:.2f}")


# Measure the magnitude of harmonics (integer multiples of the dominant
# frequency) for periodicity analysis
harmonics = []
for i in range(2, 5):  # Look for the first 4 harmonics
    harmonic_frequency = dominant_frequency * i
    idx = np.argmin(np.abs(fft_frequencies - harmonic_frequency))
    harmonics.append(fft_magnitude[idx])

print(f"Harmonic Magnitudes: {harmonics}")
